In [16]:
import numpy as np
import pandas as pd
import itertools
from Bio import SeqIO
from tqdm.auto import tqdm

## Wczytanie i przygotowanie danych

### Pliki `.bed`

In [17]:
negative_set = pd.read_csv('negative_set.bed', delimiter='\t', header=None, names=["Chromosome", "StartPos", "EndPos"])
positive_set = pd.read_csv('GM12878.bed', delimiter='\t', header=None, names=["Chromosome", "StartPos", "EndPos", "Score"])

In [18]:
negative_set.head(5)

,Chromosome,StartPos,EndPos
0,chr10,32251,36771
1,chr10,39431,39891
2,chr10,72312,74222
3,chr10,84717,85177
4,chr10,90499,91949


In [19]:
positive_set.head(5)

,Chromosome,StartPos,EndPos,Score
0,chr1,773300,774100,7.866088
1,chr1,778980,779450,6.472419
2,chr1,800100,802000,11.010675
3,chr1,825670,826410,6.114487
4,chr1,839470,842590,8.848865


In [20]:
# dodanie etykiety do zbiorów
negative_set["IsEnhancer"] = 0
positive_set["IsEnhancer"] = 1

positive_set = positive_set.drop(columns="Score")

In [21]:
# połączenie zbiorów danych
total_set = pd.concat([negative_set, positive_set])

# usunięcie nieprawidłowych chromosomów
total_set = total_set.loc[total_set["Chromosome"].str.match(r"chr\d+")]

total_set = total_set.reset_index(drop=True)

### Plik `.fasta`

In [22]:
# odczyt pliku za pomocą biblioteki biopython

fasta_sequences = {}

with open("GRCh37.primary_assembly.genome.fa", "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        fasta_sequences[record.id] = record.seq

In [23]:
# mamy podział na chromosomy
list(fasta_sequences.keys())[:5]

['chr1', 'chr10', 'chr11', 'chr12', 'chr13']

In [24]:
# w każdym chromosomie mamy sekwencję DNA
fasta_sequences["chr1"]

Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')

In [25]:
# na podstawie danych z plików .bed
# wybieramy odpowiednie fragmenty DNA (chromosom, pozycja początkowa i końcowa)

dna_sequence_list = []

for index, data in total_set.iterrows():
    chromosome = data["Chromosome"]
    start_pos = data["StartPos"]
    end_pos = data["EndPos"]
    dna_sequence = str(fasta_sequences[chromosome][start_pos:end_pos])
    dna_sequence_list.append(dna_sequence)

total_set["DNA sequence"] = dna_sequence_list

In [26]:
total_set.head(5)

,Chromosome,StartPos,EndPos,IsEnhancer,DNA sequence
0,chr10,32251,36771,0,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...
1,chr10,39431,39891,0,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...
2,chr10,72312,74222,0,CTGCCACCGTGCCTGGCTAATTTCTGTATTTTTAGTAGAGAGGGGG...
3,chr10,84717,85177,0,AGAAGTTGAATCTCTGAATAGACCAATAACAGGATCTGAAATTGTG...
4,chr10,90499,91949,0,AGGTCAGGTGATCTGCAGCCAACCAAGCAGCTGCTAAGTGGCCAAC...


In [27]:
# usuwanie rekordów z 'N' w sekwencji DNA
total_set = total_set[~total_set["DNA sequence"].str.contains("N")]

### k-mery

In [28]:
# funkcja do zliczenia k-merów w sekwencji DNA
# dla całej ramki total_set
# i do zapisu zliczonych częstości do pliku
def count_and_save_kmers(k):

    # lista wszystkich k-merów
    kmers = [''.join(kmer_tuple) for kmer_tuple in itertools.product('ACTG', repeat=k)]

    # funkcja do odwracania k-merów
    def reverse_kmer(seq: str):
        mapping = {
            "A": "T",
            "T": "A",
            "C": "G",
            "G": "C",
        }
        return "".join([mapping[s] for s in seq[::-1]])

    # lista unikalnych k-merów, które (i których odwrócenia) będziemy zliczać
    # dla k=4 liczy 136 elementów
    unique_kmers = []
    for kmer in kmers:
        reversed_kmer = reverse_kmer(kmer)
        if kmer not in unique_kmers and reversed_kmer not in unique_kmers:
            unique_kmers.append(kmer)

    # słownik do zliczania k-merów
    kmers_counts = {kmer: pd.Series(index=total_set.index, data=0) for kmer in unique_kmers}

    # zliczamy k-mery
    for kmer in tqdm(kmers):
        if kmer in unique_kmers:  
            kmers_counts[kmer] += total_set["DNA sequence"].str.count(kmer)
        else:
            reversed_kmer = reverse_kmer(kmer)
            kmers_counts[reversed_kmer] = total_set["DNA sequence"].str.count(kmer)

    # łączymy częstości w ramkę danych
    kmers_counts_df = pd.concat(kmers_counts, axis=1).round(6)

    # dzielimy przez długość sekwencji
    kmers_counts_df = kmers_counts_df.div(total_set["DNA sequence"].str.len(), axis=0)

    # łączymy z pierwotną ramką total_set i zapisujemy do pliku 
    total_set_with_kmers = pd.concat([total_set, kmers_counts_df], axis=1)
    (
        total_set_with_kmers
            .drop(columns=["StartPos", "EndPos", "DNA sequence"])
            .to_parquet(f"total_set_k_mers/total_set_{k}_mer.gzip", compression="gzip", index=False)
    )

In [ ]:
# zliczamy częstości k-merów dla kilku wybranych k

# for k in [2, 3, 4, 5]:
#     count_and_save_kmers(k)

## Przygotowanie klasyfikatora

### Wczytanie zbiorów z policzonymi częstościami k-merów

In [32]:
# wczytanie i podział na zbiory treningowy i testowy

k = 5
total_set = pd.read_parquet(f"total_set_k_mers/total_set_{k}_mer.gzip")

test_set = total_set.loc[total_set["Chromosome"].isin(["chr1", "chr14", "chr21"])]
test_set_X = test_set.drop(columns=["Chromosome", "IsEnhancer"])
test_set_Y = test_set["IsEnhancer"]

trening_set = total_set.loc[~total_set["Chromosome"].isin(["chr1", "chr14", "chr21"])]
trening_set_X = trening_set.drop(columns=["Chromosome", "IsEnhancer"])
trening_set_Y = trening_set["IsEnhancer"]